In [1]:
# Attempt to train an mnist model and distill it into a smaller model
# conclusion, distillation is a scam, at least for mnist
# works on CIFAR! at least a little

In [2]:
%pylab inline
import os
import time
import random
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = [x/255.0 for x in [x_train, x_test]]
y_train, y_test = [tf.keras.utils.to_categorical(x) for x in [y_train, y_test]]

Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_m(scale=64, loss='categorical_crossentropy'):
    x = in1 = Input((32,32,3))

    x = Conv2D(scale, (3,3), padding='same')(x)
    x = ELU()(x)
    x = Conv2D(scale, (3,3), padding='same')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(2*scale, (3,3), padding='same')(x)
    x = ELU()(x)
    x = Conv2D(2*scale, (3,3), padding='same')(x)
    x = ELU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(512)(x)
    x = ELU()(x)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)

    m = Model(in1, x)
    m.compile('adam', loss, metrics=['accuracy'])
    return m

In [4]:
m = get_m(32)
m.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
elu (ELU)                    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
elu_1 (ELU)                  (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496 

In [5]:
m.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=25)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 9s 181us/sample - loss: 1.4052 - accuracy: 0.5020 - val_loss: 1.1197 - val_accuracy: 0.6124
Epoch 2/25
50000/50000 [==============================] - 5s 91us/sample - loss: 1.0535 - accuracy: 0.6350 - val_loss: 1.0033 - val_accuracy: 0.6538
Epoch 3/25
50000/50000 [==============================] - 5s 92us/sample - loss: 0.9297 - accuracy: 0.6784 - val_loss: 0.9338 - val_accuracy: 0.6824
Epoch 4/25
50000/50000 [==============================] - 5s 94us/sample - loss: 0.8382 - accuracy: 0.7088 - val_loss: 0.8846 - val_accuracy: 0.6943
Epoch 5/25
50000/50000 [==============================] - 5s 93us/sample - loss: 0.7549 - accuracy: 0.7354 - val_loss: 0.9046 - val_accuracy: 0.6943
Epoch 6/25
50000/50000 [==============================] - 5s 94us/sample - loss: 0.6684 - accuracy: 0.7657 - val_loss: 0.8685 - val_accuracy: 0.7118
Epoch 7/25
50000/50000 [==============================]

In [6]:
rv = m.predict(x_train)

In [7]:
sm1 = get_m(16)
sm2 = get_m(16)
sm1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
elu_5 (ELU)                  (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
elu_6 (ELU)                  (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        4640

In [8]:
sm1.fit(x_train, rv, validation_data=(x_test, y_test), batch_size=128, epochs=10)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 6s 125us/sample - loss: 1.4395 - accuracy: 0.4919 - val_loss: 1.1357 - val_accuracy: 0.6114
Epoch 2/10
50000/50000 [==============================] - 5s 94us/sample - loss: 1.1053 - accuracy: 0.6166 - val_loss: 1.0645 - val_accuracy: 0.6317
Epoch 3/10
50000/50000 [==============================] - 5s 91us/sample - loss: 0.9900 - accuracy: 0.6567 - val_loss: 0.9986 - val_accuracy: 0.6600
Epoch 4/10
50000/50000 [==============================] - 5s 94us/sample - loss: 0.9267 - accuracy: 0.6835 - val_loss: 0.9554 - val_accuracy: 0.6734
Epoch 5/10
50000/50000 [==============================] - 5s 93us/sample - loss: 0.8745 - accuracy: 0.6989 - val_loss: 0.9285 - val_accuracy: 0.6810
Epoch 6/10
50000/50000 [==============================] - 4s 89us/sample - loss: 0.8285 - accuracy: 0.7137 - val_loss: 0.9187 - val_accuracy: 0.6867
Epoch 7/10
50000/50000 [==============================]

In [9]:
sm2.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=10)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 6s 126us/sample - loss: 1.4627 - accuracy: 0.4831 - val_loss: 1.1588 - val_accuracy: 0.6005
Epoch 2/10
50000/50000 [==============================] - 4s 89us/sample - loss: 1.0950 - accuracy: 0.6196 - val_loss: 1.0425 - val_accuracy: 0.6417
Epoch 3/10
50000/50000 [==============================] - 5s 92us/sample - loss: 0.9938 - accuracy: 0.6565 - val_loss: 0.9906 - val_accuracy: 0.6576
Epoch 4/10
50000/50000 [==============================] - 5s 92us/sample - loss: 0.9347 - accuracy: 0.6761 - val_loss: 0.9704 - val_accuracy: 0.6699
Epoch 5/10
50000/50000 [==============================] - 5s 90us/sample - loss: 0.8806 - accuracy: 0.6935 - val_loss: 0.9296 - val_accuracy: 0.6851
Epoch 6/10
50000/50000 [==============================] - 5s 93us/sample - loss: 0.8321 - accuracy: 0.7114 - val_loss: 0.9343 - val_accuracy: 0.6793
Epoch 7/10
50000/50000 [==============================]